# Dialogue and Narrative Coursework - Subtask 1 - New Dataset

In [ ]:
## imports
%%capture
!pip install datasets

from datasets import load_dataset, Dataset
import pandas as pd

In [ ]:
## A function to load a specific doc2dial dataset

def load_doc2dial_dataset(name='dialogue_domain', split='train'):
  cache_dir = "./data_cache"

  return load_dataset(
      "doc2dial",
      name=name,
      split=split,
      ignore_verifications=True,
      cache_dir=cache_dir,
  )


In [ ]:
train_dialogues = load_doc2dial_dataset(name="dialogue_domain", split="train")
val_dialogues = load_doc2dial_dataset(name="dialogue_domain", split="validation")

documents = load_doc2dial_dataset(name="document_domain", split="train")

train_data = load_doc2dial_dataset(name="doc2dial_rc", split="train")
val_data = load_doc2dial_dataset(name="doc2dial_rc", split="validation")

Downloading:   0%|          | 0.00/3.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.88M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

...

Dataset doc2dial downloaded and prepared to ./data_cache/doc2dial/document_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c. Subsequent calls will reuse this data.


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset doc2dial downloaded and prepared to ./data_cache/doc2dial/doc2dial_rc/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c. Subsequent calls will reuse this data.


Reusing dataset doc2dial (./data_cache/doc2dial/doc2dial_rc/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)


In [ ]:
train_data[0]

{'answers': {'answer_start': [346],
  'text': ['you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. ']},
 'context': 'Many DMV customers make easily avoidable mistakes that cause them significant problems, including encounters with law enforcement and impounded vehicles. Because we see customers make these mistakes over and over again , we are issuing this list of the top five DMV mistakes and how to avoid them. \n\n1. Forgetting to Update Address \nBy statute , you must report a change of address to DMV within ten days of moving. That is the case for the address associated with your license, as well as all the addresses associated with each registered vehicle, which may differ. It is not sufficient to only: write your new address on the back of your old license; tell the United States Postal Service; or inform the poli

In [ ]:
def add_spans(sample, dialogues):
  dial_id, turn_id = sample['id'].split('_')

  dial = dialogues.filter(lambda ex: ex['dial_id'] == dial_id)[0]

  doc_id = dial['doc_id']   ## corresponding document id
  span_ids = []             ## corresponding span ids

  for tr in dial['turns']:
    if tr['turn_id'] == int(turn_id)+1:

      for sp in tr['references']:
        span_ids.append(sp['sp_id'])

      break

  doc = documents.filter(lambda ex: ex['doc_id'] == doc_id)[0]
  
  spans = {}
  answer_spans = {}
  for span in doc['spans']:
    spans[span['id_sp']] = span['text_sp']
    if span['id_sp'] in span_ids:
      answer_spans[span['id_sp']] = span['text_sp']

  sample['spans'] = spans
  sample['answers']['spans'] = answer_spans

  return sample

In [ ]:
from tqdm import tqdm

new_train_data = pd.DataFrame(columns=['id', 'question', 'context', 'answers', 'spans',  'domain', 'title'])

for ex in tqdm(train_data):
  new_train_data = new_train_data.append(add_spans(ex, train_dialogues), ignore_index=True)

new_train_data.head()

  0%|          | 0/20431 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 1/20431 [00:02<14:18:17,  2.52s/it]Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c/cache-97c73a4132fe7d79.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c/cache-fe750ef1cc2bb737.arrow
Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c/cache-97c73a4132fe7d79.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c/cache-fe750ef1cc2bb737.arrow
Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c/cache-97c73a4132fe7d79.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.

  0%|          | 0/4 [00:00<?, ?ba/s]

...

  0%|          | 0/4 [00:00<?, ?ba/s]

Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c/cache-8233e1a1196f4098.arrow
  1%|          | 110/20431 [00:33<1:46:34,  3.18it/s]Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c/cache-4ce36a6b1d3be1fa.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c/cache-8233e1a1196f4098.arrow
Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c/cache-4ce36a6b1d3be1fa.arrow
Loading cached processed dataset at ./data_cache/doc2dial/document_domain/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c/cache-8233e1a1196f4098.arrow
Loading cached processed dataset at ./data_cache/doc2dial/dialogue_domain/1.0

  0%|          | 0/4 [00:00<?, ?ba/s]

  1%|          | 114/20431 [00:34<1:43:18,  3.28it/s]


KeyboardInterrupt: ignored

In [ ]:
new_train_data.to_csv('doc2dial_rc_train.csv')
# new_train_data = pd.read_csv('doc2dial_rc_train.csv')
new_train_data.head()

In [ ]:
new_train_data

In [ ]:
new_val_data = pd.DataFrame(columns=['id', 'question', 'context', 'answers', 'spans',  'domain', 'title'])

for ex in tqdm(val_data):
  new_val_data = new_val_data.append(add_spans(ex, val_dialogues), ignore_index=True)

new_val_data.head()

In [ ]:
new_val_data.to_csv('doc2dial_rc_val.csv')